https://julialang.org/blog/2017/08/dsl/<br>
**Creating domain-specific languages in Julia using macros**<br>
在Julia中使用宏创建特定于领域的语言

9 August 2017 | David P. Sanders <br>
8月9日 | 大卫·桑德斯<br>

Since the beginning of Julia, it has been tempting to use macros to write domain-specific languages (DSLs), i.e. to extend Julia syntax to provide a simpler interface to create Julia objects with complicated behaviour. The first, and still most extensive, example is JuMP.<br>

自Julia诞生之初，人们就开始尝试使用宏来编写领域特定语言（dsl），即扩展Julia语法以提供更简单的接口来创建具有复杂行为的Julia对象。第一个也是最广泛的例子是JuMP。

Since the fix for the infamous early Julia issue #265, which was incorporated in Julia 0.6, some previous methods for creating DSLs in Julia, mainly involving eval, ceased to work.<br>

由于修复了臭名昭著的早期Julia问题#265，它被合并到Julia 0.6中，以前在Julia中创建dsl的一些方法，主要涉及 eval ，停止工作了。<br>

In this post, we will describe a recommended pattern (i.e., a reusable structure) for creating DSLs without the use of eval, using syntax suitable for Julia 0.6 and later versions; it is strongly recommended to upgrade to Julia 0.6.<br>

在这篇文章中，我们将描述一个推荐的模式（即，一个可重用的结构），用于创建dsl而不使用 eval ，使用适合Julia 0.6和更高版本的语法；强烈建议升级到Julia 0.6。

### Creating a Model object containing a function<br>
创建包含函数的 Model 对象<br>

This blog post arose from a question in the JuliaCon 2017 hackathon about the Modia modelling language, where there is a `@model` macro. Here we will describe the simplest possible version of such a macro, which will create a `Model` object that contains a function, and is itself callable.<br>

这篇博文源于2017年JuliaCon黑客马拉松上关于Modia建模语言的一个问题，其中有一个 `@model` 宏。这里我们将描述这样一个宏的最简单的可能版本，它将创建一个 `Model` 对象，该对象包含一个函数，并且它本身是可调用的。<br>

First we define the `Model` object. It is tempting to write it like this:<br>
首先定义 `Model` 对象。这样写很有诱惑力：

In [2]:
struct NaiveModel
    f::Function
end

We can then create an instance of the `NaiveModel` type (i.e., an object of that type) using the default constructor, e.g. by passing it an anonymous function:<br>

然后，我们可以使用默认构造函数创建一个 `NaiveModel` 类型的实例（即该类型的对象），例如通过传递一个匿名函数：

In [3]:
m1 = NaiveModel(x -> 2x)

NaiveModel(var"#13#14"())

and we can call the function using<br>
我们可以用

In [4]:
m1.f(10)

20

If we wish instances like `m` to themselves behave like functions, we can overload the call syntax on the `NaiveModel` object:<br>

如果希望像 `m` 这样的实例本身表现得像函数，可以在 `NaiveModel` 对象上重载调用语法：

In [5]:
(m::NaiveModel)(x) = m.f(x)

so that we can now just write<br>
现在我们可以这样写

In [6]:
m1(10)

20

### Parametrising the type 参数化类型
Since `Function` is an abstract type, for performance we should not have a field of this type inside our object. Rather, we parametrise the type using the type of the function:<br>

由于 `Function` 是一个抽象类型，为了性能考虑，不应该在对象中使用该类型的字段。相反，我们使用函数的类型来参数化类型：

In [7]:
struct Model{F}
    f::F
end

(m::Model)(x) = m.f(x)

In [8]:
m2 = Model(x->2x)

Model{var"#15#16"}(var"#15#16"())

In [9]:
m2(10)

20

Let's compare the performance:<br>
让我们比较一下性能：

In [11]:
using BenchmarkTools
@btime m1(10)
@btime m2(10)

  36.895 ns (0 allocations: 0 bytes)
  18.455 ns (0 allocations: 0 bytes)


20

### Manipulating expressions 操作表达式
We wish to define a *macro* that will allow us to use a simple syntax, of our choosing, to create objects. Suppose we would like to use the syntax<br>
我们希望定义一个宏，它允许我们使用自己选择的简单语法来创建对象。假设我们希望使用该语法<br>

`julia> @model 2x`<br>

to define a `Model` object containing the function `x -> 2x`. Note that `2x` on its own is not valid Julia syntax for creating a function; the macro will allow us to use this simplified syntax for our own purposes.<br>

定义一个 `Model` 对象，包含函数 `x -> 2x` 。请注意， `2x` 本身在Julia语法中创建函数是无效的；宏将允许我们使用这种简化的语法来实现我们自己的目的。<br>

Before getting to macros, let's first build some tools to manipulate the expression `2x` in the correct way to build a `Model` object from it, using standard Julia functions.<br>

在讨论宏之前，让我们先构建一些工具，以正确的方式操作表达式 `2x` ，从而使用标准的Julia函数从中构建 `Model` 对象。<br>

First, let's create a function to manipulate our expression:<br>
首先，让我们创建一个函数来操作表达式：

In [15]:
function make_function(ex::Expr)
    return :(x -> $ex)
end

make_function (generic function with 1 method)

In [16]:
ex = :(2x)
f = make_function(ex)

:(x->begin
          #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X12sZmlsZQ==.jl:2 =#
          2x
      end)

Here, we have created a Julia expression called `ex`, which just contains the expression `2x` that we would like for the body of our new function, and we have passed this expression into `make_function`, which wraps it into a complete anonymous function. This assumes that `ex` is an expression containing the variable `x` and makes a new expression representing an anonymous function with the single argument `x`. (See e.g. my JuliaCon 2017 tutorial for an example of how to walk through the expression tree in order to extract automatically the variables that it contains.)<br>

在这里，我们创建了一个名为 `ex` 的Julia表达式，它只包含我们希望作为新函数体的表达式 `2x` ，并且我们将该表达式传递给 `make_function` ，这将其包装成一个完整的匿名函数。这里假设 `ex` 是一个包含变量 `x` 的表达式，并生成一个新表达式，表示具有单个参数 `x` 的匿名函数。（例如，请参阅我的JuliaCon 2017教程，了解如何遍历表达式树以自动提取其包含的变量的示例。）<br>

Now let's define a function `make_model` that takes a function, wraps it, and passes it into a `Model` object:<br>

现在让我们定义一个函数 `make_model` ，它接受一个函数，包装它，并把它传递给一个 `Model` 对象：

In [14]:
function make_model(ex::Expr)
    return :(Model($ex))
end

make_model (generic function with 1 method)

In [17]:

make_model(f)

:(Model((x->begin
              #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X12sZmlsZQ==.jl:2 =#
              2x
          end)))

If we evaluate this "by hand", we see that it correctly creates a `Model` object:<br>
如果我们“手工”计算，我们会看到它正确地创建了一个 `Model` 对象：

In [18]:
m3 = eval(make_model(f))

Model{var"#17#18"}(var"#17#18"())

In [19]:
m3(10)

20

### Macros 宏
However, this is ugly and clumsy. Instead, we now wrap everything inside a macro. A macro is a code manipulator: it eats code, massages it in some way (possibly including completely rewriting it), and spits out the new code that was produced. This makes macros an incredibly powerful (and, therefore, dangerous) tool when correctly used.<br>

然而，这是丑陋和笨拙的。相反，我们现在将所有内容包装在一个宏中。宏是一个代码操纵者：它吞噬代码，以某种方式（可能包括完全重写它）对其进行处理，并吐出生成的新代码。这使得宏在正确使用时成为一个非常强大的（因此也是危险的）工具。<br>

In the simplest case, a macro takes as argument a single Julia `Expr` object, i.e. an unevaluated Julia expression (i.e., a piece of Julia code). It manipulates this expression object to create a new expression object, which it then returns.<br>

在最简单的情况下，宏接受一个Julia  `Expr` 对象作为参数，即一个未求值的Julia表达式（即一段Julia代码）。它操作这个表达式对象来创建一个新的表达式对象，然后返回这个对象。<br>

The key point is that this returned expression is "spliced into" the newly-generated code in place of the old code. The compiler will never actually see the old code, only the new code.<br>

关键的一点是，这个返回的表达式被“拼接”到新生成的代码中，以取代旧代码。编译器实际上永远不会看到旧代码，只看到新代码。<br>

Let's start with the simplest possible macro:<br>
让我们从最简单的宏开始：

In [20]:
macro model(ex)
    @show ex
    @show typeof(ex)
    return nothing
end

@model (macro with 1 method)

This just shows the argument that it was passed and exits, returning an empty expression.<br>
这只显示传递给它的参数并退出，返回一个空表达式。

In [21]:
m4 = @model 2x

ex = :(2x)
typeof(ex) = Expr


We see that the Julia `Expr` object has been automatically created from the explicit code that we typed.<br>
我们看到Julia  `Expr` 对象是从我们键入的显式代码中自动创建的。<br>

Now we can plug in our previous functions to complete the macro's functionality:<br>
现在我们可以插入前面的函数来完成宏的功能：

In [22]:
macro model(ex)
    return make_model(make_function(ex))
end

@model (macro with 1 method)

In [23]:
m5 = @model 2x
m5 |> display

m5(10)

Model{var"#19#20"}(var"#19#20"())

20

To check that the macro is doing what we think it is, we can use the `@macroexpand` command, which itself is a macro (as denoted by the initial `@`):<br>

为了检查宏是否正在执行我们认为的操作，我们可以使用 `@macroexpand` 命令，该命令本身就是一个宏（由初始 `@` 表示）：

In [24]:
@macroexpand @model 2x

:(Main.Model(((var"#72#x",)->begin
              #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X12sZmlsZQ==.jl:2 =#
              2var"#72#x"
          end)))

### Macro "hygiene" 宏“卫生”
However, our macro has an issue, called macro "hygiene". This has to do with where variables are defined. Let's put everything we have so far inside a module:<br>

然而，我们的宏有一个问题，称为宏“卫生”。这与变量的定义位置有关。让我们把到目前为止所有的东西都放在一个模块中：

In [25]:
module Models

    export Model, @model

    struct Model{F}
        f::F
    end

    (m::Model)(x) = m.f(x)

    function make_function(ex::Expr)
        return :(x -> $ex)
    end

    function make_model(ex::Expr)
        return :(Model($ex))
    end

    macro model(ex)
        return make_model(make_function(ex))
    end

end

Main.Models

Now we import the module and use the macro:<br>
现在我们导入模块并使用宏：

In [26]:
using .Models
m6 = @model 2x

Model{var"#21#22"}(var"#21#22"())

In [27]:
m6(10)

20

So far so good. But now let's try to include a global variable in the expression:<br>
到目前为止一切顺利。但是现在让我们试着在表达式中包含一个全局变量：

In [28]:
a = 2
m7 = @model 2*a*x

Model{var"#23#24"}(var"#23#24"())

We see that it cannot find `a`. Let's see what the macro is doing:<br>
我们看到它不能找到 `a` 。让我们看看宏在做什么：<br>

**（笔者这没问题, 引用了Main.a, 原作者的Julia版本可能是0.6,版本间语法有变化）**

In [29]:
@macroexpand @model 2*a*x

:(Main.Model(((var"#77#x",)->begin
              #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X12sZmlsZQ==.jl:2 =#
              2 * Main.a * var"#77#x"
          end)))

In [30]:
m7(10)

40

```julia
julia> @macroexpand @model 2*a*x
:((Models.Model)((#4#x->begin  # In[1], line 12:
                    2 * Models.a * #4#x
                end)))
```
We see that Julia is looking for `Models.a`, i.e. a variable `a` defined inside the `Models` module.<br>
我们看到Julia正在寻找 `Models.a` ，即在 `Models` 模块中定义的变量 `a` 。<br>

To fix this problem, we must write an "unhygienic" macro, by "escaping" the code, using the `esc` function. This is a mechanism telling the compiler to look for variable definitions in the scope from which the macro is called (here, the current module `Main`), rather than the scope where the macro is defined (here, the `Models` module):<br>

为了解决这个问题，我们必须编写一个“不卫生”的宏，通过使用 `esc` 函数“转义”代码。这是一种机制，告诉编译器在调用宏的作用域中（这里，当前模块 `Main` ）寻找变量定义，而不是在定义宏的作用域中（这里， `Models` 模块）：

In [31]:
module Models2

export Model, @model

struct Model{F}
    f::F
end

(m::Model)(x) = m.f(x)

function make_function(ex::Expr)
    return :(x -> $ex)
end

function make_model(ex::Expr)
    return :(Model($ex))
end

macro model(ex)
    return make_model(make_function(esc(ex)))
end

end

Main.Models2

In [32]:
using .Models2
a = 2
m8 = @model 2*a*x

Model{var"#25#26"}(var"#25#26"())

In [33]:
@macroexpand @model 2*a*x

:(Main.Model(((var"#81#x",)->begin
              #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X12sZmlsZQ==.jl:2 =#
              2 * Main.a * var"#81#x"
          end)))

In [34]:
m8(10)

40

This is the final, working version of the macro.<br>
这是该宏的最终工作版本。

### Conclusion 结论
We have successfully completed our task: we have seen how to create a macro that enables a simple syntax for creating a Julia object that we can use later.<br>
我们已经成功地完成了我们的任务：我们已经看到了如何创建一个宏，该宏支持一种简单的语法来创建我们以后可以使用的Julia对象。

For some more in-depth discussion of metaprogramming techniques and macros, **see my video tutorial Invitation to intermediate Julia, given at JuliaCon 2016:**<br>

有关元编程技术和宏的更深入讨论，请参阅我在2016年JuliaCon上发布的视频教程《邀请中级Julia》：<br>

link to the [video](https://www.youtube.com/watch?v=rAxzR7lMGDM)<br>
link to the [Jupyter notebooks](https://github.com/dpsanders/intermediate_julia)<br>

Author: David P. Sanders, Associate Professor, Department of Physics, Faculty of Sciences, National University of Mexico (UNAM).